# SQL - 데이터 전체 조회

In [7]:
import sqlite3

conn = sqlite3.connect('member.db')
cursor = conn.cursor()

#데이터조회 SQL문 실행
cursor.execute("select * from meminfo")
rows = cursor.fetchall()#모든 레코드 가져오기

for row in rows:
    print(row)
    
conn.close()

('wynam', 'n1234', '남운영', '2019-08-20 13:40:50')
('jklee', 'l3456', '이전경', '2019-08-20 14:50:10')


# SQL - 열 이름으로 조회

In [10]:
import sqlite3
conn = sqlite3.connect('member.db')

conn.row_factory = sqlite3.Row
cursor = conn.cursor()
cursor.execute('select * from meminfo')#meminfo 테이블 내 모든 데이터
rows = cursor.fetchall()#rows에 레코드 저장

for row in rows:
    print("ID : %s, PASSWD : %s, name : %s, time : %s" %
          (row['id'], row['passwd'], row['name'], row['time']))
    
for row in rows:
    print(row)

conn.close()

ID : wynam, PASSWD : n1234, name : 남운영, time : 2019-08-20 13:40:50
ID : jklee, PASSWD : l3456, name : 이전경, time : 2019-08-20 14:50:10


# 기상청 데이터 응용

In [41]:
import urllib.request
from bs4 import BeautifulSoup
#기상청 날씨페이지 로드
page = urllib.request.urlopen('http://www.weather.go.kr/weather/\
observation/currentweather.jsp').read()
soup = BeautifulSoup(page, "html.parser")

#DB 연결
import sqlite3
conn = sqlite3.connect('Weather.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [42]:
#현재시간 태그 검색
current_time = soup.find('input',{'class':'time'})
#날짜 시간 분리 후 SQLite3 형식의 날짜 시간 문자열로 변환
dt = current_time['value'].split('.')
year, month, date, time = dt[0], dt[1], dt[2], dt[3]
date_time = year + "-" + month + "-" + date + " " + time
print(date_time)

2021-10-05 22:00


In [43]:
#현재 날씨테이블 검색
table = soup.find('table', {'class':'table_develop3'})
data = []

for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    for td in tds:
        if td.find('a'):
            city = td.find('a').text
            temperature = tds[5].text
            humidity = tds[9].text
            data.append([city, temperature, humidity])
            #레코드 삽입 SQL문 실행
            cursor.execute('''
            insert into CityWeather (city, temperature, humidity, dtime) values (?,?,?,?)''',
                           (city, temperature, humidity, date_time))
            
conn.commit()#데이터베이스 내보내기
#데이터 조회 SQL문 실행, 31로 검색 안되서 20로 변경
cursor.execute("select * from CityWeather where temperature > 20")
rows = cursor.fetchall()

for row in rows:
    print("도시 : %s, 온도 : %s, 습도 : %s, 시간 : %s"%
         (row['city'],row['temperature'], row['humidity'], row['dtime']))
    
conn.close()

도시 : 강진군, 온도 : 21.5, 습도 : 93, 시간 : 2021-10-05 22:00
도시 : 거제, 온도 : 21.8, 습도 : 92, 시간 : 2021-10-05 22:00
도시 : 경주시, 온도 : 20.8, 습도 : 91, 시간 : 2021-10-05 22:00
도시 : 고산, 온도 : 24.2, 습도 : 93, 시간 : 2021-10-05 22:00
도시 : 고흥, 온도 : 20.3, 습도 : 98, 시간 : 2021-10-05 22:00
도시 : 광양시, 온도 : 23.0, 습도 : 88, 시간 : 2021-10-05 22:00
도시 : 광주, 온도 : 22.9, 습도 : 93, 시간 : 2021-10-05 22:00
도시 : 군산, 온도 : 20.8, 습도 : 99, 시간 : 2021-10-05 22:00
도시 : 김해시, 온도 : 23.0, 습도 : 88, 시간 : 2021-10-05 22:00
도시 : 남해, 온도 : 21.3, 습도 : 94, 시간 : 2021-10-05 22:00
도시 : 대구, 온도 : 21.7, 습도 : 82, 시간 : 2021-10-05 22:00
도시 : 대전, 온도 : 21.3, 습도 : 90, 시간 : 2021-10-05 22:00
도시 : 목포, 온도 : 22.9, 습도 : 82, 시간 : 2021-10-05 22:00
도시 : 밀양, 온도 : 21.4, 습도 : 84, 시간 : 2021-10-05 22:00
도시 : 보령, 온도 : 20.5, 습도 : 94, 시간 : 2021-10-05 22:00
도시 : 보성군, 온도 : 21.5, 습도 : 95, 시간 : 2021-10-05 22:00
도시 : 부산, 온도 : 23.2, 습도 : 88, 시간 : 2021-10-05 22:00
도시 : 북창원, 온도 : 23.3, 습도 : 89, 시간 : 2021-10-05 22:00
도시 : 서귀포, 온도 : 24.8, 습도 : 81, 시간 : 2021-10-05 22:00
도시 : 서산, 온도 : 21.4, 습도 :

# 임의의 프로그램